In [32]:
%pip install yfinance

In [33]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

#pd.set_option('display.max_rows', 500)

In [34]:
nextDays = 100
def createDataFrame(stock, start, end):
    df = stock.history(start=start, end=end)

    df["Change"] = df["Close"].pct_change() # 전날 대비 변화율

    # add next 2 days' average
    #df["Next2"] = df["Close"].rolling(window=2).mean().shift(-2)

    # add next 5 days' average
    df["Next"] = df["Close"].rolling(window=nextDays).mean().shift(-nextDays)

    # add 10 days later
    df["Next"] = (df["Close"].shift(-nextDays+1) + df["Close"].shift(-nextDays) + df["Close"].shift(-nextDays-1)) / 3




    # df.index = pd.to_datetime(df.index).strftime("%Y-%m-%d") # 날짜 형식 간단하게 만들기

    # 지표 데이터
    # df2 = pd.read_csv("/Users/ryujonghyeok/Jonghyeok/Sejong/SAI/자료모음집.csv")
    # df = pd.merge(df, df2, on="Date") """

    # df.set_index("Date", inplace=True)

    return df

In [35]:
# Define the stock
stock = yf.Ticker("SPY")

# Create a dataframe from 'start' to 'end'
start = "2010-01-01"
end = "2023-12-31"

df = createDataFrame(stock, start, end)
df = df[["Close", "Change", "Next"]]
df

,Close,Change,Next
Date,,,
2010-01-04 00:00:00-05:00,86.860092,NaN,83.962392
2010-01-05 00:00:00-05:00,87.090004,0.002647,84.054741
2010-01-06 00:00:00-05:00,87.151321,0.000704,83.944430
2010-01-07 00:00:00-05:00,87.519188,0.004221,84.288185
2010-01-08 00:00:00-05:00,87.810463,0.003328,84.106051
...,...,...,...
2023-12-22 00:00:00-05:00,472.182892,0.002010,NaN
2023-12-26 00:00:00-05:00,474.176697,0.004223,NaN
2023-12-27 00:00:00-05:00,475.034058,0.001808,NaN


In [36]:
# FEDFUNDS: Federal Funds Effective Rate
# DFEDTARU: Federal Funds Target Range - Upper Limit
# CPIAUCSL: Consumer Price Index for All Urban Consumers: All Items in U.S. City Average
# DGS10: Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity, Quoted on an Investment Basis


# access CPI.csv file in 'data' folder
cpiaucsl = pd.read_csv('https://github.com/ryujonghyeok-student/TraderJohns/blob/main/data/CPIAUCSL.csv?raw=true')
dfedtaru = pd.read_csv('https://github.com/ryujonghyeok-student/TraderJohns/blob/main/data/DFEDTARU.csv?raw=true')
dgs10 = pd.read_csv('https://github.com/ryujonghyeok-student/TraderJohns/blob/main/data/DGS10.csv?raw=true')
fedfunds = pd.read_csv('https://github.com/ryujonghyeok-student/TraderJohns/blob/main/data/FEDFUNDS.csv?raw=true')
unrate = pd.read_csv('https://github.com/ryujonghyeok-student/TraderJohns/blob/main/data/UNRATE.csv?raw=true')



# convert 'DATE' column to datetime
cpiaucsl['DATE'] = pd.to_datetime(cpiaucsl['DATE'])
dfedtaru['DATE'] = pd.to_datetime(dfedtaru['DATE'])
dgs10['DATE'] = pd.to_datetime(dgs10['DATE'])
fedfunds['DATE'] = pd.to_datetime(fedfunds['DATE'])
unrate['DATE'] = pd.to_datetime(unrate['DATE'])

cpiaucsl.set_index('DATE', inplace=True)
dfedtaru.set_index('DATE', inplace=True)
dgs10.set_index('DATE', inplace=True)
fedfunds.set_index('DATE', inplace=True)
unrate.set_index('DATE', inplace=True)

# set start and end date with dataframe data type
start = pd.to_datetime(start)
end = pd.to_datetime(end)

# combine cpiaucsl, dfedtaru, dgs10, fedfunds, unrate with key being index
data = pd.concat([cpiaucsl, dfedtaru, dgs10, fedfunds, unrate], axis=1)
data = data[start:end]

data = data.replace(".", np.NaN)
data = data.ffill()

data

,CPIAUCSL,DFEDTARU,DGS10,FEDFUNDS,UNRATE
DATE,,,,,
2010-01-01,217.488,0.25,NaN,0.11,9.8
2010-01-02,217.488,0.25,NaN,0.11,9.8
2010-01-03,217.488,0.25,NaN,0.11,9.8
2010-01-04,217.488,0.25,3.85,0.11,9.8
2010-01-05,217.488,0.25,3.77,0.11,9.8
...,...,...,...,...,...
2023-12-27,308.742,5.50,3.79,5.33,3.7
2023-12-28,308.742,5.50,3.84,5.33,3.7
2023-12-29,308.742,5.50,3.88,5.33,3.7


In [37]:
# Convert 'df' to tz-aware dataframe
df = df.tz_localize(None)

# Combine 'df' and 'data' dataframe respect to 'DATE' as index
df = pd.merge(df, data, left_index=True, right_index=True, how='inner')

df

,Close,Change,Next,CPIAUCSL,DFEDTARU,DGS10,FEDFUNDS,UNRATE
2010-01-04,86.860092,NaN,83.962392,217.488,0.25,3.85,0.11,9.8
2010-01-05,87.090004,0.002647,84.054741,217.488,0.25,3.77,0.11,9.8
2010-01-06,87.151321,0.000704,83.944430,217.488,0.25,3.85,0.11,9.8
2010-01-07,87.519188,0.004221,84.288185,217.488,0.25,3.85,0.11,9.8
2010-01-08,87.810463,0.003328,84.106051,217.488,0.25,3.83,0.11,9.8
...,...,...,...,...,...,...,...,...
2023-12-22,472.182892,0.002010,NaN,308.742,5.50,3.90,5.33,3.7
2023-12-26,474.176697,0.004223,NaN,308.742,5.50,3.89,5.33,3.7
2023-12-27,475.034058,0.001808,NaN,308.742,5.50,3.79,5.33,3.7
2023-12-28,475.213501,0.000378,NaN,308.742,5.50,3.84,5.33,3.7


In [38]:
# import the scikit learn linear regression model
from sklearn.linear_model import LinearRegression

# create a linear regression model between 'Close' and 'CPIAUCSL', 'DFEDTARU', 'DGS10', 'FEDFUNDS', 'UNRATE
model = LinearRegression()

# x = df[['CPIAUCSL', 'DFEDTARU', 'DGS10', 'FEDFUNDS', 'UNRATE']]
x = df.iloc[:-nextDays-1][['CPIAUCSL', 'DFEDTARU', 'DGS10', 'FEDFUNDS', 'UNRATE']]

# y = df['Close']
y = df.iloc[:-nextDays-1]['Next']


model.fit(x, y) # train the model

# print the model coefficients
print(model.coef_)
print(model.intercept_)
print(model.score(x, y))

""" # add predicted 'Close' values to the dataframe
df['Predicted Close'] = model.predict(x)

# plot the 'Close' and 'Predicted Close' values
plt.plot(df.index, y, label='Close')
plt.plot(df.index, model.predict(x), label='Predicted Close')
plt.legend()
plt.show() """

[  4.91403452 -30.60582854 -27.9333031   26.39760552   0.55990785]
-916.3670227257655
0.9239964478279261


" # add predicted 'Close' values to the dataframe\ndf['Predicted Close'] = model.predict(x)\n\n# plot the 'Close' and 'Predicted Close' values\nplt.plot(df.index, y, label='Close')\nplt.plot(df.index, model.predict(x), label='Predicted Close')\nplt.legend()\nplt.show() "

In [39]:
def buy(현금보유량, 주식보유량, 현재주가):
    if 주식보유량 == 0:
        주식보유량 = 현금보유량 // 현재주가
        현금보유량 -= 주식보유량 * 현재주가
        # print(f"Bought at {현재주가} with {주식보유량} stocks")
    return 현금보유량, 주식보유량


def sell(현금보유량, 주식보유량, 현재주가):
    if 주식보유량 > 0:
        현금보유량 += 주식보유량 * 현재주가
        주식보유량 = 0
        # print(f"Sold at {현재주가} with {주식보유량} stocks")
    return 현금보유량, 주식보유량

def total_asset(현금보유량, 주식보유량, 현재주가):
    return 현금보유량 + 주식보유량 * 현재주가

In [40]:
buyAndHold_plt = []
# 시작시점에 매수 후 종료시점에 매도
def strategy_buyAndHold(df):
    global buyAndHold_plt

    현금보유량 = 1000000
    주식보유량 = 0

    현금보유량, 주식보유량 = buy(현금보유량, 주식보유량, df.iloc[0].Close)

    buyAndHold_plt = []
    for i in range(1, len(df)-1):
        buyAndHold_plt.append(total_asset(현금보유량, 주식보유량, df.iloc[i].Close))

    현금보유량 += 주식보유량 * df.iloc[-1].Close

    print(f"Hold - 현금보유량: {현금보유량:.2f}, profit: {(현금보유량-1000000)/1000000*100:.2f}%")


In [41]:
difference_plt = []
# 최고점 대비 일정 비율 하락하면 매도, 최저점 대비 일정 비율 상승하면 매수
def strategy_difference(df):
    global difference_plt
    현금보유량 = 1000000
    주식보유량 = 0
    dif1 = 0.36 # 일정 비율 하락하는 정도 🤑
    dif2 = 0.02 # 일정 비율 상승하는 정도 🤑

    minimum = float('inf')
    maximum = float('-inf')

    for i in range(2, len(df)):
        price = df.iloc[i].Close
        if price > maximum:
            maximum = price
        elif price < minimum:
            minimum = price
        if 주식보유량 > 0 and price < maximum * (1-dif1):
            print("DIF - Bought")
            현금보유량, 주식보유량 = sell(현금보유량, 주식보유량, price)
            maximum = float('-inf')
        elif 주식보유량 == 0 and price > minimum * (1+dif2):
            #print(f"Bought at {price(hist, i)}")
            현금보유량, 주식보유량 = buy(현금보유량, 주식보유량, price)
            minimum = float('inf')

        difference_plt.append(total_asset(현금보유량, 주식보유량, price))

    현금보유량 += 주식보유량 * df.iloc[-1].Close
    주식보유량 = 0

    print(f"Difference - 현금보유량: {현금보유량:.2f}, profit: {(현금보유량-1000000)/1000000*100:.2f}%")

In [42]:
""" import requests
import datetime
import pandas as pd
from bs4 import BeautifulSoup

requests.packages.urllib3.disable_warnings()

def scraper(url):
    resp = requests.get(url, verify=False)
    page = resp.text
    soup = BeautifulSoup(page, "html.parser")

    body_copy = str(soup.find_all("p", {"class": "body-copy"}))
    body_copy = body_copy.replace('<p class="body-copy">', '')
    body_copy = body_copy.replace('</p>', '')

    return body_copy


for i in range(1, len(df)):
    a = df.iloc[i].name
    a = a.strftime("%b-%d-%Y")
    a = str(a).lower()
    url = f"https://www.investrade.com/market-review-{a}/"
    content = scraper(url)
    df.loc[df.index[i], "Title"] = content

df.tail()

 """

' import requests\nimport datetime\nimport pandas as pd\nfrom bs4 import BeautifulSoup\n\nrequests.packages.urllib3.disable_warnings()\n\ndef scraper(url):\n    resp = requests.get(url, verify=False)\n    page = resp.text\n    soup = BeautifulSoup(page, "html.parser")\n\n    body_copy = str(soup.find_all("p", {"class": "body-copy"}))\n    body_copy = body_copy.replace(\'<p class="body-copy">\', \'\')\n    body_copy = body_copy.replace(\'</p>\', \'\')\n\n    return body_copy\n\n\nfor i in range(1, len(df)):\n    a = df.iloc[i].name\n    a = a.strftime("%b-%d-%Y")\n    a = str(a).lower()\n    url = f"https://www.investrade.com/market-review-{a}/"\n    content = scraper(url)\n    df.loc[df.index[i], "Title"] = content\n\ndf.tail()\n\n '

In [43]:
def score_calculate_og(i):
    # 4.76130047  -2.86062042 -31.53250228  -8.93819761  -6.0887131
    # x = df[['CPIAUCSL', 'DFEDTARU', 'DGS10', 'FEDFUNDS', 'UNRATE']]

    d_cpiaucsl = float(df.iloc[i]["CPIAUCSL"])
    d_dfedtaru = float(df.iloc[i]["DFEDTARU"])
    d_dgs10 = float(df.iloc[i]["DGS10"])
    d_fedfunds = float(df.iloc[i]["FEDFUNDS"])
    d_unrate = float(df.iloc[i]["UNRATE"])

    # [  3.31071085   5.16950057  -2.57965745 -19.2770091  -34.5469385 ]
    # -414.4216677959783

    score = 4.76130047 * d_cpiaucsl - 2.86062042 * d_dfedtaru - 31.53250228 * d_dgs10 - 8.93819761 * d_fedfunds - 6.0887131 * d_unrate -838.334901756302
    score = 3.31071085 * d_cpiaucsl + 5.16950057 * d_dfedtaru - 2.57965745 * d_dgs10 - 19.2770091 * d_fedfunds - 34.5469385 * d_unrate -414.4216677959783

    # [  5.87635666 -20.95913652   8.94047031  19.06006088   2.26623342]
    # -1252.8649401375355
    score = 5.87635666 * d_cpiaucsl - 20.95913652 * d_dfedtaru + 8.94047031 * d_dgs10 + 19.06006088 * d_fedfunds + 2.26623342 * d_unrate -1252.8649401375355

    return score

def score_calculate(i):
    # [  4.33353229  -2.0621398   -5.18311157 -15.21641093 -46.91571719]
    # -609.9466426959245


    d_cpiaucsl = float(df.iloc[i]["CPIAUCSL"])
    d_dfedtaru = float(df.iloc[i]["DFEDTARU"])
    d_dgs10 = float(df.iloc[i]["DGS10"])
    d_fedfunds = float(df.iloc[i]["FEDFUNDS"])
    d_unrate = float(df.iloc[i]["UNRATE"])

    # [  5.69925028 -48.076492     1.56366513  53.42014896   4.19152599]
    # -1199.3135901021014

    score = 4.33353229 * d_cpiaucsl - 2.0621398 * d_dfedtaru - 5.18311157 * d_dgs10 - 15.21641093 * d_fedfunds - 46.91571719 * d_unrate -609.9466426959245
    score = 5.87635666 * d_cpiaucsl - 20.95913652 * d_dfedtaru + 8.94047031 * d_dgs10 + 19.06006088 * d_fedfunds + 2.26623342 * d_unrate -1252.8649401375355
    score = 5.55300228 * d_cpiaucsl - 32.33691206 * d_dfedtaru + 4.82001601 * d_dgs10 + 36.57436438 * d_fedfunds + 2.97079608 * d_unrate -1170.3482155841334
    score = 5.44129951 * d_cpiaucsl - 21.80079886 * d_dfedtaru + 8.01323566 * d_dgs10 + 24.88876588 * d_fedfunds + 1.66806938 * d_unrate -1147.2719098050673 # Result of SPY 20090101-20201231 and to the avg of 9, 10, 11th day
    score = 5.69925028 * d_cpiaucsl - 48.076492 * d_dfedtaru + 1.56366513 * d_dgs10 + 53.42014896 * d_fedfunds + 4.19152599 * d_unrate -1199.3135901021014 # Result of SPY20090101-20201231 and to the avg of 99, 100, 101st day

    return score


score_plt = []
# 복합적인 요소를 분석하여 일정 조건에 따라 매수 및 매도 반복
def strategy_score(df):
    global score_plt

    현금보유량 = 1000000
    주식보유량 = 0

    현금보유량, 주식보유량 = buy(현금보유량, 주식보유량, df.iloc[0].Close)

    for i in range(len(df)):

        # Calculate the initial score
        score = score_calculate(i)

        현재주가 = df.iloc[i].Close



        # 점수가 일정 기준보다 클 때 매수하기
        # If x in "score * x" is greater than 1, then it is much easier to buy the stock
        if  현재주가 < score * 1.2:
            현금보유량, 주식보유량 = buy(현금보유량, 주식보유량, 현재주가)

        # 점수가 일정 기준보다 작을 때 매도하기
        # if x  in "score * x" is greater than 1, then it is much harder to sell the stock
        elif 현재주가 > score * 1.05:
            현금보유량, 주식보유량 = sell(현금보유량, 주식보유량, 현재주가)

        score_plt.append(total_asset(현금보유량, 주식보유량, 현재주가))

    현금보유량, 주식보유량 = sell(현금보유량, 주식보유량, df.iloc[-1].Close)


    print(f"Score V1 - 현금보유량: {현금보유량:.2f} profit: {(현금보유량-1000000)/1000000*100:.2f}%")


In [44]:
score2_plt = []
def strategy_score2(df):
    global score2_plt
    현금보유량 = 1000000
    주식보유량 = 0
    현금보유량, 주식보유량 = buy(현금보유량, 주식보유량, df.iloc[0].Close)


    # Adaption of difference-strategy

    # 일정 비율 하락하는 정도
    dif1 = 0.2 # Parameter 1
    # 일정 비율 상승하는 정도
    dif2 = 0.05 # Parameter 2

    minimum = float('inf')
    maximum = float('-inf')

    for i in range(len(df)):

        # Calculate the initial score
        score = score_calculate(i)

        현재주가 = df.iloc[i].Close


        price = df.iloc[i].Close
        if price > maximum:
            maximum = price
        elif price < minimum:
            minimum = price
        elif 주식보유량 == 0 and price > minimum * (1+dif2): # Rise so Buy
            score *= 1.4 # Parameter 3
            minimum = float('inf')
        elif 주식보유량 > 0 and price < maximum * (1-dif1): # Drop so Sell
            score * 0.9 # Parameter 4
            maximum = float('-inf')


        # 점수가 일정 기준보다 클 때 매수하기
        # If x in "score * x" is greater than 1, then it is much easier to buy the stock
        if  현재주가 < score * 1.0: # Parameter 5
            현금보유량, 주식보유량 = buy(현금보유량, 주식보유량, 현재주가)

        # 점수가 일정 기준보다 작을 때 매도하기
        # if x  in "score * x" is greater than 1, then it is much harder to sell the stock
        elif 현재주가 > score * 1.0: # Parameter 6
            현금보유량, 주식보유량 = sell(현금보유량, 주식보유량, 현재주가)

        score2_plt.append(total_asset(현금보유량, 주식보유량, 현재주가))

    현금보유량, 주식보유량 = sell(현금보유량, 주식보유량, df.iloc[-1].Close)


    print(f"Score V2 - 현금보유량: {현금보유량:.2f} profit: {(현금보유량-1000000)/1000000*100:.2f}%")

In [ ]:
def strategy_score2_test(df, dif1, dif2, score_multiplier1, score_multiplier2):

    현금보유량 = 1000000
    주식보유량 = 0
    현금보유량, 주식보유량 = buy(현금보유량, 주식보유량, df.iloc[0].Close)

    minimum = float('inf')
    maximum = float('-inf')

    for i in range(len(df)):

        # Calculate the initial score
        score = score_calculate(i)

        현재주가 = df.iloc[i].Close


        price = df.iloc[i].Close
        if price > maximum:
            maximum = price
        elif price < minimum:
            minimum = price
        elif 주식보유량 == 0 and price > minimum * (1+dif2): # Rise so Buy
            score *= score_multiplier1 # Parameter 3
            minimum = float('inf')
        elif 주식보유량 > 0 and price < maximum * (1-dif1): # Drop so Sell
            score * score_multiplier2 # Parameter 4
            maximum = float('-inf')


        # 점수가 일정 기준보다 클 때 매수하기
        # If x in "score * x" is greater than 1, then it is much easier to buy the stock
        if  현재주가 < score: # Parameter 5
            현금보유량, 주식보유량 = buy(현금보유량, 주식보유량, 현재주가)

        # 점수가 일정 기준보다 작을 때 매도하기
        # if x  in "score * x" is greater than 1, then it is much harder to sell the stock
        elif 현재주가 > score: # Parameter 6
            현금보유량, 주식보유량 = sell(현금보유량, 주식보유량, 현재주가)


    현금보유량, 주식보유량 = sell(현금보유량, 주식보유량, df.iloc[-1].Close)


    print(f"{dif1}, {dif2}, {score_multiplier1}, {score_multiplier2} -> {(현금보유량-1000000)/1000000*100:.2f}%")


l_dif = [0.1, 0.15, 0.2, 0.25, 0.3]
l_mul = [0.8, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15, 1.2]

for i in l_dif:
    if i == 0.1 or i == 0.15:
        continue
  for j in l_dif:
    for k in l_mul:
      for l in l_mul:
        strategy_score2_test(df, i, j, k, l)


0.1, 0.1, 0.8, 0.8 -> 481.87%
0.1, 0.1, 0.8, 1.0 -> 481.87%
0.1, 0.1, 0.8, 1.2 -> 481.87%
0.1, 0.1, 1.0, 0.8 -> 472.57%
0.1, 0.1, 1.0, 1.0 -> 472.57%
0.1, 0.1, 1.0, 1.2 -> 472.57%


In [ ]:
strategy_buyAndHold(df)
strategy_difference(df)
strategy_score(df)
strategy_score2(df)

# 그래프 그리기
plt.figure(figsize=(20, 10))
plt.plot(df.index[:-2], buyAndHold_plt, label="buyAndHold")
plt.plot(df.index[:-2], difference_plt, label="difference")
plt.plot(df.index, score_plt, label="score")
plt.plot(df.index, score2_plt, label="score2")
plt.legend()
plt.show()